In [ ]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print("Project root is set:", project_root)


Projektwurzel gesetzt auf: /home/homeflo/workspace/project_trading/Trading-Bot


## === Step 1: Load data  ===

In [ ]:
symbol = "GBP_USD_M1_08-01-2025_09-01-2025_with_volume.pkl"  #"XAUUSD_H1.csv"
print(f"📊 Loading data for {symbol} with days_back=60...")
freq = "15M"
# Use the same data loading function
from core.data_loader import load_and_resample
resampled = load_and_resample(f"../data/{symbol}", days_back=60)
data = resampled.get(freq)

if data is None or data.empty:
    print(f"❌ ERROR: No data loaded for the '{freq}' timeframe.")
    
else:
    print(f"📊 Loaded {len(data)} '{freq}' candles from {data.index.min()} to {data.index.max()}")

🚀 Starting BOS/CHOCH Backtester
📊 Loading data for GBP_USD_M1_08-01-2025_09-01-2025_with_volume.pkl with days_back=60...
📊 Loaded 1959 '15M' candles from 2025-08-01 00:00:00+00:00 to 2025-08-31 23:45:00+00:00


## === Step 2: Analyse market structure ===

In [ ]:
print("🔍 Analyzing market structure...")
from core.structure_builder import get_market_analysis
analysis = get_market_analysis(data, prominence_factor=2.5)
structure = analysis['structure']

print(f"📊 Total structure points: {len(structure)}")


🔍 Analyzing market structure...
📊 Total structure points: 61


## === Step 3: Detect market events ===

## === Step 4: Initialize and run backtrader ===

## === Step 5: Run backtest ===

## === Step 6: Display results ===

## === Step 7: Generate visualizations ===